In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import numpy as np
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

# from utils import make_or_load_subjects_electrodes_to_ROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance

import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import gee
from statsmodels.genmod import families

# from src.analysis.power.roi_analysis import blah_blah
from src.analysis.config import experiment_conditions
from src.analysis.config.plotting_parameters import plotting_parameters as plot_params
import src.analysis.utils.general_utils as utils # import utils functions one by one by name
from src.analysis.power.power_traces import make_multi_channel_evokeds_for_all_conditions_and_rois, plot_power_traces_for_all_rois, create_subtracted_evokeds_dict 

['/hpc/home/etb28/miniconda3/envs/ieeg/lib/python311.zip', '/hpc/home/etb28/miniconda3/envs/ieeg/lib/python3.11', '/hpc/home/etb28/miniconda3/envs/ieeg/lib/python3.11/lib-dynload', '', '/hpc/home/etb28/miniconda3/envs/ieeg/lib/python3.11/site-packages']
Qt5Agg backend not available, using default backend
Qt5Agg backend not available, using default backend


In [2]:
## set lab LAB_root

LAB_root = None
task='GlobalLocal'
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

LAB_root = '/cwork/etb28/'


# Get data layout
layout = get_data(task, root=LAB_root)
save_dir_root = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

print("setting path")

setting path


In [3]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
# subjects = ['D0057']

In [4]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = os.path.join(project_root, 'src', 'analysis', 'config')

subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                        filename='subjects_electrodestoROIs_dict.json')

print("subjects electrodes to rois dict")

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from /hpc/group/coganlab/etb28/GlobalLocal/src/analysis/config/subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!
subjects electrodes to rois dict


In [5]:
# Calculate the total number of electrodes in the 'default_dict' across subjects
total_electrodes = utils.count_electrodes_across_subjects(subjects_electrodestoROIs_dict, subjects)
print('total elecs:', total_electrodes)

total elecs: 3283


In [6]:
combined_data = pd.read_csv(os.path.join(LAB_root, 'D_Data', 'GlobalLocal', 'combinedData.csv'))
combined_data[['switchProportion', 'congruencyProportion']] = combined_data.apply(utils.map_block_type, axis=1)

print("load behavioral")

load behavioral


In [7]:
task='GlobalLocal'
conditions = experiment_conditions.stimulus_err_congruency_conditions # set this to whichever conditions you're running
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = [condition for condition in conditions.keys()]

print("load epochs")

load epochs


In [8]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_err_corr_conditions:
    conditions_save_name = 'stimulus_err_corr_conditions_sem_green' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_err_congruency_conditions:
    conditions_save_name = 'stimulus_err_congruency_conditions_sem' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_err_corr_conditions:
    conditions_save_name = 'response_err_corr_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_err_congruency_conditions:
    conditions_save_name = 'response_err_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
    
# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
#subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True, error_trials_only=False)
#one without selecting only correct trials
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=False)

Loading data for subject: D0057
Reading /cwork/etb28/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/D0057/D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading /cwork/etb28/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/D0057/D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_power_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection it

/hpc/group/coganlab/etb28/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_err_con: 19 valid trials out of 19
  Loading condition: Stimulus_err_inc
Not setting metadata
13 matching events found
No baseline correction applied


/hpc/group/coganlab/etb28/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_err_inc: 13 valid trials out of 13
  Loading condition: Stimulus_err_con
Not setting metadata
19 matching events found
No baseline correction applied


...

    Stimulus_err_inc: 19 valid trials out of 19
  Loading condition: Stimulus_err_con
Not setting metadata
15 matching events found
No baseline correction applied


/hpc/group/coganlab/etb28/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_err_con: 15 valid trials out of 15
  Loading condition: Stimulus_err_inc
Not setting metadata
19 matching events found
No baseline correction applied


/hpc/group/coganlab/etb28/GlobalLocal/src/analysis/utils/general_utils.py:442: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs_list)


    Stimulus_err_inc: 19 valid trials out of 19
